In [2]:
%pip install --quiet \ pandas 

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: Directory '\\' is not installable. Neither 'setup.py' nor 'pyproject.toml' found.


In [ ]:
#carregar dados

import pandas as pd
import pandas as pd
from geopy.distance import geodesic
import folium
from IPython.display import display

# Carregar dados binários de produtos
produtos_df = pd.read_csv('../data/frutas_olericolas_binario.csv')
produtos_df = produtos_df.replace({'V': True, 'F': False})
produtos_df['Cidade'] = produtos_df['Cidade'].str.strip().str.lower()

# Carregar dados das cooperativas
cooperativas_df = pd.read_csv('../data/cooperativas.csv')
cooperativas_df['Região'] = cooperativas_df['Região'].str.lower().str.strip()

C:\Users\Gustavo Borges\AppData\Local\Temp\ipykernel_5800\394122892.py:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  produtos_df = produtos_df.replace({'V': True, 'F': False})


EmptyDataError: No columns to parse from file

In [7]:
dados = [
    ("associação de Agricultores Familiares da Eco Comunidade do Assentamento 15 de Agosto (Afeca)", ["são sebastião"], -15.91117477568832, -47.72142914924957),

    ("Associação Agrícola do Distrito Federal e Ride (Agrifam)", ["taguatinga", "vargem bonita"], -15.831, -48.056),

    ("Associação Mista dos Agricultores Familiares, Orgânicos e Produtores Rurais do DF e Entorno (Amista)", ["gama"], -15.794, -47.930),

    ("Associação dos Produtores da Agricultura Familiar (Aspaf )", ["gama"], -15.61, -48.2),

    ("Associação dos Produtores Rurais de Alexandre Gusmão (AsSpag)", ["brazlândia", "alexandre gusmão"], -15.747628314703643, -48.11713041048713),

    ("Associação dos Produtores de Hortigranjeiros do Distrito Federal (Asphor)", ["gama"], -15.791489057677724, -47.948277524935406),


    ("Associação dos Produtores do Núcleo Rural Lago Oeste (ASPROESTE)", ["sobradinho"],-15.619109134078176, -47.94872798840174),
    
    ("Associação dos Produtores Rurais Novo Horizonte Betinho (Aspronte)", ["ceilândia"], -15.603462102101528, -48.11337371660706),

    ("Associação dos Trabalhadores Rurais da Agricultura Familiar do Assentamento Chapadinha (Astraf)", ["Sobradinho"], -15.542745521391486, -48.02980085045159),


    ("Cooperativa Mista da Agricultura Familiar do Meio Ambiente e da Cultura do Brasil", ["gama"], -15.94, -48.02),

    ("Cooperativa Agrícola Buriti Vermelho (Cooper-Horti)",["paranoá","pad-df", "jardim"], -15.900840116171455, -47.41563678134716),

    ("Cooperativa dos Produtores Rurais de Planaltina de Goiás e Região (Prorural)", ["paranoá", "pad-df", "jardim"], -15.445611703837635, -47.6190618576721),

    ("Cooperativa Agropecuária da Região de Brazlândia (Coopebraz)", ["brazlândia", "taguatinga", "alexandre gusmão","vargem bonita"], -15.60313666754234, -48.11359575477609),

    ("Cooperativa de Agricultura Familiar Mista do Distrito Federal (Coopermista)", ["planaltina","pipiripau", "rio preto", "taquara", "tabatinga","vargem bonita"], -15.764378385278865, -47.4937971768077),

    ("Cooperativa dos Agricultores Familiares Ecológicos do Cerrado (RedeTerra)", ["gama"], -15.794, -47.930),
    ("Cooperativa Agrícola da Região de Planaltina (Cootaquara) ",["ceilândia"],-16.014, -48),

    ("Cooperativa de Serviços Ambientais, Agricultura Familiar, Sociedade,Cultura e Saúde (CooperBrasilia)" , ["sobradinho", "são sebastião"], -15.65, -47.8),

    ("Cooperativa Agroambiental Palmas do Lago Oeste (CooperPalmas)", ["sobradinho", "planaltina", "pipiripau", "rio preto", "taquara", "tabatinga", "vargem bonita"], -15.572147049026993, -48.0142951696709),

]
csv_file = '../data/cooperativas.csv'
with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Nome", "Região", "Latitude", "Longitude"])
    for nome, regioes, lat, lon in dados:
        writer.writerow([nome, ', '.join(regioes), lat, lon])




NameError: name 'csv' is not defined

In [39]:
#Sistema de recomendação

from geopy.distance import geodesic

class RecomendadorAgricolaDF:
    def __init__(self, produtos_df, cooperativas_df):
        self.produtos = produtos_df
        self.cooperativas = cooperativas_df.explode('Região')
        
    def recomendar(self, produtos_alvo, origem, raio_km=20):
        """
        Args:
            produtos_alvo: Lista de produtos (ex: ['Tomate', 'Alface'])
            origem: Tupla (latitude, longitude)
            raio_km: Raio de busca em quilômetros
            
        Returns:
            DataFrame com cooperativas que produzem TODOS os produtos no raio especificado
        """
        # 1. Encontrar cidades com todos os produtos desejados
        cidades_validas = self.produtos[
            self.produtos[produtos_alvo].all(axis=1)
        ]['Cidade'].unique()
        
        # 2. Filtrar cooperativas nessas cidades
        cooperativas_validas = self.cooperativas[
            self.cooperativas['Região'].isin(cidades_validas)
        ].copy()
        
        # 3. Calcular distâncias
        cooperativas_validas['Distância (km)'] = cooperativas_validas.apply(
            lambda x: int(round(geodesic(origem, (x['Latitude'], x['Longitude'])).km)),
            axis=1
        )
        
        # 4. Filtrar por raio e ordenar
        return cooperativas_validas[
            cooperativas_validas['Distância (km)'] <= raio_km
        ].sort_values('Distância (km)')

In [32]:
#gera mapa
def visualizar_mapa(self, resultados, origem):
        """Gera mapa interativo com os resultados"""
        mapa = folium.Map(location=origem, zoom_start=10)
        
        # Marcador da origem
        folium.Marker(
            location=origem,
            popup='<b>Sua Localização</b>',
            icon=folium.Icon(color='blue', icon='home')
        ).add_to(mapa)
        
        # Marcadores das cooperativas
        for _, row in resultados.iterrows():
            folium.Marker(
                location=[row['Latitude'], row['Longitude']],
                popup=f"""
                <b>{row['Nome']}</b><br>
                Região: {row['Região']}<br>
                Distância: {row['Distância (km)']}km<br>
                Produtos: {row['Produtos']}
                """,
                icon=folium.Icon(color='green', icon='leaf')
            ).add_to(mapa)
        
        # Linhas de conexão
        for _, row in resultados.iterrows():
            folium.PolyLine(
                locations=[origem, [row['Latitude'], row['Longitude']]],
                color='gray',
                weight=1
            ).add_to(mapa)
        
        return mapa

In [40]:
#teste de saída
# Exemplo de Uso
if __name__ == "__main__":
    sistema = RecomendadorAgricolaDF(produtos_df, cooperativas_df)
    
    # Exemplo: Buscar produtores de tomate e alface próximo ao Plano Piloto
    resultados = sistema.recomendar(
        produtos_alvo=['Tangerina'],
        origem=(-15.794, -47.882),  # Coordenadas do Plano Piloto
        raio_km=30
    )
    
    print("🍅🌱 Top Recomendações:")
    display(resultados)
    
    print("\n🗺️ Mapa Interativo:")
    mapa = sistema.visualizar_mapa(resultados, (-15.794, -47.882))
    display(mapa)

🍅🌱 Top Recomendações:


,Nome,Região,Latitude,Longitude,Distância (km)
2,"Associação Mista dos Agricultores Familiares, ...",gama,-15.794000,-47.930000,5
14,Cooperativa dos Agricultores Familiares Ecológ...,gama,-15.794000,-47.930000,5
5,Associação dos Produtores de Hortigranjeiros d...,gama,-15.791489,-47.948278,7
6,Associação dos Produtores do Núcleo Rural Lago...,sobradinho,-15.619109,-47.948728,21
0,associação de Agricultores Familiares da Eco C...,são sebastião,-15.911175,-47.721429,22
9,Cooperativa Mista da Agricultura Familiar do M...,gama,-15.940000,-48.020000,22
15,Cooperativa Agrícola da Região de Planaltina (...,ceilândia,-16.014000,-48.000000,27



🗺️ Mapa Interativo:


AttributeError: 'RecomendadorAgricolaDF' object has no attribute 'visualizar_mapa'